In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import tensorflow as tf
import os
from tqdm import tqdm
import cv2
from PIL import Image
from joblib import Parallel, delayed
import gc
from PIL import Image as im
import random
from scipy import ndimage


In [2]:

#!pip install /kaggle/input/rsnapacks/dicomsdl-0.109.1-cp36-cp36m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
#import pydicom as dicom

import tensorflow as tf
from PIL import Image as im
import os
from tqdm import tqdm
import cv2
from PIL import Image
from joblib import Parallel, delayed
import gc
import random

In [3]:
with (open('/kaggle/input/processed/imgDataNew.pkl', "rb")) as openfile:
     imgData = pickle.load(openfile)
imgData=imgData[:]

In [4]:
print(imgData['cancer'].value_counts())

0    42838
1    11580
Name: cancer, dtype: int64


In [5]:



train_target = np.array(imgData['cancer'])
train_features = np.array(list(imgData['img_data_process']))


gc.collect()


0

In [6]:
def normalize(image):
    # Repeat channels to create 3 channel images required by pretrained ConvNextV2 models
    #image = tf.repeat(image, repeats=3, axis=3)
    # Cast to float 32
    image = tf.cast(image, tf.float32)
    # Normalize with respect to ImageNet mean/std
    image = tf.keras.applications.imagenet_utils.preprocess_input(image, mode='torch')

    return image

In [7]:
INPUT_SHAPE=(95,95,3)
!pip install --no-deps /kaggle/input/kerasmodel/keras_cv_attention_models-1.3.5-py3-none-any.whl
from keras_cv_attention_models import convnext
#resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
#tf.config.experimental_connect_to_cluster(resolver)
#tf.tpu.experimental.initialize_tpu_system(resolver)
#tpu_strategy = tf.distribute.experimental.TPUStrategy(resolver)
#with tpu_strategy.scope():
def get_model():
        # Inputs, note the names are equal to the dictionary keys in the dataset
        image = tf.keras.layers.Input(INPUT_SHAPE, name='image', dtype=tf.uint8)

        # Normalize Input
        image_norm = normalize(image)

        # CNN Feature Maps
        x = convnext.ConvNeXtV2Tiny(
            input_shape=(95, 95, 3),
            pretrained=None,
            num_classes=0,
        )(image_norm)

        # Average Pooling BxHxWxC -> BxC
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        # Dropout to prevent Overfitting
        x = tf.keras.layers.Dropout(0.30)(x)
        # Output value between [0, 1] using Sigmoid function
        outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

        # Define model with inputs and outputs
        model = tf.keras.models.Model(inputs=image, outputs=outputs)

        # Load pretrained Model Weights
        #model.load_weights('/kaggle/input/rsna-efficientnetv2-training-tensorflow-tpu-ds/model.h5')

        # Set model non-trainable
        model.trainable = False

        # Compile model
        model.compile()

        return model

Processing /kaggle/input/kerasmodel/keras_cv_attention_models-1.3.5-py3-none-any.whl


In [8]:
# Pretrained File Path: '/kaggle/input/sartorius-training-dataset/model.h5'
tf.keras.backend.clear_session()
# enable XLA optmizations
tf.config.optimizer.set_jit(True)

model = get_model()
model.compile(\
        loss="binary_crossentropy",\
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),\
        metrics=["acc"])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image (InputLayer)           [(None, 95, 95, 3)]       0         
_________________________________________________________________
tf.cast (TFOpLambda)         (None, 95, 95, 3)         0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 95, 95, 3)         0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 95, 95, 3)         0         
_________________________________________________________________
tf.math.truediv_1 (TFOpLambd (None, 95, 95, 3)         0         
_________________________________________________________________
convnext_v2_tiny (Functional (None, 2, 2, 768)         27864960  
_________________________________________________________________
global_average_pooling2d (Gl (None, 768)               0     

In [9]:
weights = {0:1, 1:3.5}
model.fit(train_features, train_target, class_weight=weights, validation_split=0.2,batch_size=16,epochs=150)


Epoch 1/150
2721/2721 [==============================] - 92s 25ms/step - loss: 1.3240 - acc: 0.3689 - val_loss: 0.9706 - val_acc: 0.2891
Epoch 2/150
2721/2721 [==============================] - 72s 27ms/step - loss: 1.3217 - acc: 0.3676 - val_loss: 0.9706 - val_acc: 0.2891
Epoch 3/150
2721/2721 [==============================] - 61s 22ms/step - loss: 1.3258 - acc: 0.3662 - val_loss: 0.9706 - val_acc: 0.2891
Epoch 4/150
2721/2721 [==============================] - 66s 24ms/step - loss: 1.3242 - acc: 0.3652 - val_loss: 0.9706 - val_acc: 0.2891
Epoch 5/150
2721/2721 [==============================] - 64s 24ms/step - loss: 1.3204 - acc: 0.3679 - val_loss: 0.9706 - val_acc: 0.2891
Epoch 6/150
2721/2721 [==============================] - 62s 23ms/step - loss: 1.3213 - acc: 0.3661 - val_loss: 0.9706 - val_acc: 0.2891
Epoch 7/150
2721/2721 [==============================] - 62s 23ms/step - loss: 1.3207 - acc: 0.3657 - val_loss: 0.9706 - val_acc: 0.2891
Epoch 8/150
2721/2721 [==================

In [10]:
model.save("/kaggle/working/trained_model_breast_cancer3.h5")


/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [11]:
#from tensorflow import keras
#savedModel = keras.models.load_model("/kaggle/input/pre-trained-model-of-breast-cancer/trained_model_breast_cancer3.h5")



In [12]:
#test_datagen = ImageDataGenerator(
#    rescale=1./25)
#    featurewise_center=True,
#    featurewise_std_normalization=True)
#test_datagen.fit(test_features)

#test_generator = test_datagen.flow(test_features, batch_size=1)
#pred = model.predict(test_features)
#bin_pred = []
#for i in pred:
#    if i>=0.5:
#        bin_pred.append(1)
#    else:
#        bin_pred.append(0)
#bin_pred = np.array(bin_pred)


In [13]:
#import sklearn
#acc = sklearn.metrics.accuracy_score(test_target, bin_pred)
#print(acc)

#countzero=0
#countone=0
#countzerot=0
#countonet=0
#for i, j in zip(test_target, bin_pred):
#    if i==0 and j==0:
#        countzero+=1
#    if i==1 and j==1:
#        countone+=1
#    if i==1:
#        countonet+=1
#    if i==0:
#        countzerot+=1
#print(countzero/countzerot,countone/countonet)

In [14]:

def recall_m(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)))
    possible_positives = np.sum(np.round(np.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives )
    return recall

def precision_m(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)))
    predicted_positives = np.sum(np.round(np.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives )
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))


#print(f1_m(test_target,bin_pred))